# 04 — Model Comparison (Senaryo Odaklı)

**Amaç:** Sistematik deneme-yanılma ile leakage'sız feature set bulmak, **Reconstructed Features** ve **SMOTE** ile F1'i 0.30+ seviyelerine çekmek.

**Akış:** 1) Veri yükleme & pre-check → 2) Senaryolar A–D (Arınma) → 3) Feature Reconstruction (E) → 4) SMOTE (F) → 5) Threshold optimization & Confusion Matrix.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, f1_score, precision_score, recall_score,
    roc_curve, precision_recall_curve, confusion_matrix
)
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

warnings.filterwarnings("ignore")
RANDOM_STATE = 42
N_SPLITS = 5
np.random.seed(RANDOM_STATE)

## 1. Data loading & pre-check

`marketing_analytics_featured.csv` dosyasını yüklüyoruz. Sütunları listeliyor ve hedef değişkenin (**Conversion**) neden zorlayıcı olduğunu istatistiksel olarak açıklıyoruz.

In [2]:
path_data = "../data/marketing_analytics_featured.csv"
if not os.path.exists(path_data):
    raise FileNotFoundError(f"Veri dosyası bulunamadı: {path_data}")

df = pd.read_csv(path_data)
y = df["Conversion"]
pct_pos = y.mean() * 100
n_pos, n_neg = y.sum(), (y == 0).sum()

print("Shape:", df.shape)
print("Sütunlar:", list(df.columns))
print("\nConversion oranı (pozitif sınıf):", f"{pct_pos:.2f}%")
print("Pozitif örnek:", int(n_pos), "| Negatif örnek:", n_neg, "| Oran (neg/pos):", f"{n_neg/max(n_pos,1):.1f}")
print("\nNeden zor? ~%1–2 conversion ile sınıflar çok dengesiz; model varsayılan 0.5 eşiğinde çoğunluğu (negatif) tahmin eder, F1/recall düşer.")
df.head()

Shape: (48000, 37)
Sütunlar: ['CustomerID', 'Age', 'Gender', 'Income', 'CampaignChannel', 'CampaignType', 'AdSpend', 'ClickThroughRate', 'ConversionRate', 'WebsiteVisits', 'PagesPerVisit', 'TimeOnSite', 'SocialShares', 'EmailOpens', 'EmailClicks', 'PreviousPurchases', 'LoyaltyPoints', 'AdvertisingPlatform', 'AdvertisingTool', 'Conversion', 'CPA_Proxy', 'ROI_Proxy', 'Spend_Efficiency', 'Site_Engagement', 'Avg_Time_Per_Page', 'CTR_to_Conversion', 'Email_Click_Rate', 'Social_Virality', 'Age_Group', 'Income_Tier', 'Loyalty_Tier', 'Customer_Value_Score', 'AdSpend_x_CTR', 'Income_x_Loyalty', 'Age_x_Purchases', 'Channel_Performance', 'Is_Best_Channel']

Conversion oranı (pozitif sınıf): 2.04%
Pozitif örnek: 980 | Negatif örnek: 47020 | Oran (neg/pos): 48.0

Neden zor? ~%1–2 conversion ile sınıflar çok dengesiz; model varsayılan 0.5 eşiğinde çoğunluğu (negatif) tahmin eder, F1/recall düşer.


,CustomerID,Age,Gender,Income,CampaignChannel,CampaignType,AdSpend,ClickThroughRate,ConversionRate,WebsiteVisits,...,Social_Virality,Age_Group,Income_Tier,Loyalty_Tier,Customer_Value_Score,AdSpend_x_CTR,Income_x_Loyalty,Age_x_Purchases,Channel_Performance,Is_Best_Channel
0,8001,26,Female,53664,Affiliate,Consideration,1820.99,0.1901,0.0172,18,...,1.842105,Adult,VeryHigh,Gold,15971.14814,346.170199,18.192096,130,Low,0
1,8002,26,Female,23671,Social Media,Retention,1785.04,0.0000,0.0344,36,...,0.594595,Adult,Low,Bronze,0.00000,0.000000,0.160963,0,Medium,0
2,8003,51,Male,25799,SEO,Awareness,1775.15,0.0539,0.0227,22,...,0.173913,Senior,Low,Silver,0.00000,95.680585,1.455064,0,Low,0
3,8004,26,Female,56751,PPC,Awareness,1711.18,0.0958,0.0057,33,...,1.588235,Adult,VeryHigh,Silver,0.00000,163.931044,2.360842,0,Medium,0
4,8005,28,Female,26553,Email,Conversion,1760.34,0.2466,0.0580,34,...,1.628571,Adult,Low,Bronze,0.00000,434.099844,0.159318,0,High,1


## 2. Incremental scenario testing (A–D)

Farklı leakage seviyelerini test etmek için **evaluate_scenarios(X, y)** kullanıyoruz. Bu fonksiyon:
- Her senaryoda atılacak sütunları `prepare_X(df, drop_cols)` ile çıkarıp one-hot encode ediyor.
- **Stratified K-Fold CV** ile Imputer → StandardScaler → (isteğe SMOTE) → Logistic Regression (`class_weight='balanced'`) eğitiyor.
- ROC-AUC, F1 ve Recall ortalamalarını döndürüyor.

**Senaryolar:** A = sadece ConversionRate + CTR_to_Conversion atıldı. B = A + ROI_Proxy. C = A + CPA_Proxy. D = hepsi atıldı (dürüst ama bilgi kaybı).


In [3]:
# Senaryo tanımları: her biri atılacak sütun listesi (CustomerID, Conversion her zaman atılır)
BASE_DROP = ["CustomerID", "Conversion"]
SCENARIOS = {
    "A": BASE_DROP + ["ConversionRate", "CTR_to_Conversion"],
    "B": BASE_DROP + ["ConversionRate", "CTR_to_Conversion", "ROI_Proxy"],
    "C": BASE_DROP + ["ConversionRate", "CTR_to_Conversion", "CPA_Proxy"],
    "D": BASE_DROP + ["ConversionRate", "CTR_to_Conversion", "ROI_Proxy", "CPA_Proxy"],
}

In [4]:
def prepare_X(data, drop_cols):
    """drop_cols içindeki sütunları çıkarır, kategorikleri one-hot encode eder."""
    to_drop = [c for c in drop_cols if c in data.columns]
    X = data.drop(columns=to_drop, errors="ignore")
    X = pd.get_dummies(X, drop_first=True)
    for c in X.select_dtypes(include=["bool"]).columns:
        X[c] = X[c].astype(int)
    return X


In [5]:
def evaluate_scenarios(X, y, use_smote=False):
    """
    Cross-validation ile senaryoyu değerlendirir.
    - Imputer (NaN) → Scaler → isteğe SMOTE → Logistic Regression (class_weight='balanced').
    - use_smote=True: SMOTE sadece train fold'a uygulanır; azınlık sınıfı sentetik örneklerle artar.
    """
    imputer = SimpleImputer(strategy="median")
    scaler = StandardScaler()
    cv = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
    roc_list, f1_list, rec_list = [], [], []
    for train_idx, val_idx in cv.split(X, y):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
        X_tr = pd.DataFrame(imputer.fit_transform(X_tr), columns=X.columns, index=X_tr.index)
        X_val = pd.DataFrame(imputer.transform(X_val), columns=X.columns, index=X_val.index)
        X_tr_s = scaler.fit_transform(X_tr)
        X_val_s = scaler.transform(X_val)
        if use_smote:
            n_min = int(y_tr.sum())
            k = min(5, max(1, n_min - 1))
            X_tr_s, y_tr = SMOTE(random_state=RANDOM_STATE, k_neighbors=k).fit_resample(X_tr_s, y_tr)
        clf = LogisticRegression(class_weight="balanced", max_iter=1000, random_state=RANDOM_STATE)
        clf.fit(X_tr_s, y_tr)
        proba = clf.predict_proba(X_val_s)[:, 1]
        pred = clf.predict(X_val_s)
        roc_list.append(roc_auc_score(y_val, proba))
        f1_list.append(f1_score(y_val, pred, zero_division=0))
        rec_list.append(recall_score(y_val, pred, zero_division=0))
    return {"roc_auc": np.mean(roc_list), "f1": np.mean(f1_list), "recall": np.mean(rec_list)}


In [6]:
print("evaluate_scenarios(X, y) hazır. Senaryo A–D için sırayla çağıracağız.")


evaluate_scenarios(X, y) hazır. Senaryo A–D için sırayla çağıracağız.


In [7]:
scenario_results = []
for name, drop_cols in SCENARIOS.items():
    X_s = prepare_X(df, drop_cols)
    res = evaluate_scenarios(X_s, y, use_smote=False)
    res["scenario"] = name
    scenario_results.append(res)
scenario_df = pd.DataFrame(scenario_results).set_index("scenario")[["roc_auc", "f1", "recall"]]
print("Senaryo karşılaştırması (A–D):")
display(scenario_df.round(4))
print("En iyi F1 (A–D):", scenario_df["f1"].idxmax())

Senaryo karşılaştırması (A–D):


,roc_auc,f1,recall
scenario,,,
A,1.0000,1.0000,1.0000
B,1.0000,1.0000,1.0000
C,0.7071,0.0755,0.6306
D,0.7072,0.0760,0.6347


En iyi F1 (A–D): A


**⚠️ Leaked senaryolar:** A ve B'de F1/ROC-AUC ~1.0 çıkıyor; bu, sızıntı yapan sütunlar (ConversionRate, CTR_to_Conversion ve B'de ROI_Proxy) sayesinde modelin "cevabı görmesi" anlamına gelir. **Dürüst seçim:** En iyi senaryoyu seçerken sadece **F1 skoru 0.99'dan küçük** olan senaryoları (C, D, E, F) dikkate alacağız; böylece hileli senaryolar elenir.

### Arınma süreci: Skorlar neden 1.0'dan düşüyor?

Leakage içeren sütunlar (ConversionRate, CTR_to_Conversion, ROI_Proxy, CPA_Proxy) varken model neredeyse "cevabı görüyordu"; bu yüzden ROC-AUC ~1.0 çıkabiliyordu. Bu sütunları adım adım çıkardıkça **arınma** oluyor: model sadece gerçekten tahmin edilebilir bilgiye dayanıyor. Skorların 1.0'dan düşmesi **başarısızlık değil**, gerçekçi bir başlangıçtır. D sonrası F1 ~0.15 bandında kalabilir; Feature Reconstruction (E) ve SMOTE (F) ile bunu yükselteceğiz.